In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import DataFrame as df

from scipy import stats

import sklearn
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import KFold, RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR, SVC
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, VotingClassifier

import requests
from bs4 import BeautifulSoup as bs

In [13]:
video_df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/video/master/video_games_sale.csv',index_col=0)
video_df

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,Year_of_ten
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8.0,322.0,Nintendo,E,2000.0
1,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E,2000.0
2,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8.0,192.0,Nintendo,E,2000.0
3,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E,2000.0
4,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E,2000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6820,E.T. The Extra-Terrestrial,GBA,2001.0,Action,NewKidCo,0.01,0.00,0.00,0.00,0.01,46.0,4.0,2.4,21.0,Fluid Studios,E,2000.0
6821,Mortal Kombat: Deadly Alliance,GBA,2002.0,Fighting,Midway Games,0.01,0.00,0.00,0.00,0.01,81.0,12.0,8.8,9.0,Criterion Games,M,2000.0
6822,Metal Gear Solid V: Ground Zeroes,PC,2014.0,Action,Konami Digital Entertainment,0.00,0.01,0.00,0.00,0.01,80.0,20.0,7.6,412.0,Kojima Productions,M,2010.0
6823,Breach,PC,2011.0,Shooter,Destineer,0.01,0.00,0.00,0.00,0.01,61.0,12.0,5.8,43.0,Atomic Games,T,2010.0


video_games_sales 데이터셋(video_games_sales.csv)의 출시년도(Year_of_Release) 컬럼을 10년단위(ex 1990~1999 : 1990)로 변환하여 새로운 컬럼(year_of_ten)에 추가하고 게임이 가장 많이 출시된 년도(10년단위)와 가장 적게 출시된 년도(10년단위)를 각각 구하여라.

In [36]:
# video_df.Year_of_Release.describe()
game_sale_df = video_df.copy()
game_sale_df['year_of_ten'] = 1980

for i in range(199, 202):
    game_sale_df.loc[(game_sale_df.Year_of_Release > i*10), 'year_of_ten'] = i*10

game_sale_df.year_of_ten.value_counts()
#game_sale_df
max_decade = 2000
min_decade = 1980

플레이스테이션 플랫폼 시리즈(PS,PS2,PS3,PS4,PSV)중 장르가 Action로 발매된 게임의 총 수는?

In [47]:
ps_list = ['PS', 'PS2', 'PS3', 'PS4', 'PSV']
action_count = []
for ps in ps_list:
    df1 = game_sale_df.loc[(game_sale_df.Platform==ps)]
    df2 = df1.loc[df1.Genre=='Action']
    count = len(df2.index)
    print(ps, count)
    action_count.append(count)
print(sum(action_count))

PS 27
PS2 243
PS3 228
PS4 81
PSV 37
616


게임이 400개 이상 출시된 플랫폼들을 추출하여 각 플랫폼의 User_Score 평균값을 구하여 데이터프레임을 만들고 값을 내림차순으로 정리하여 출력하라

In [62]:
game_400s = []
for key, val in enumerate(game_sale_df.Platform.value_counts().items()):
    if val[1] >= 400:
        game_400s.append(val[0])

means = []
for ps in game_400s:
    df1 = game_sale_df.loc[(game_sale_df.Platform==ps)]
    mean_val = df1.User_Score.mean()
    means.append(mean_val)
user_score_df = df({'Platform':game_400s, 'avg_user_score':means}).sort_values('avg_user_score', ascending=False)
user_score_df

,Platform,avg_user_score
0,PS2,7.664386
4,XB,7.540708
3,PC,7.060369
6,DS,7.047198
5,Wii,6.903758
2,PS3,6.782705
1,X360,6.780769


게임 이름에 Mario가 들어가는 게임을 3회 개발한 개발자(Developer컬럼)을 구하여라

In [66]:
developer_list = []
developer_dic = {}
for i in range(len(game_sale_df.index)):
    name = str(game_sale_df.Name[i]).lower()
    if 'mario' in name:
        developer = game_sale_df.Developer[i]
        if developer not in developer_list:
            developer_list.append(developer)
            developer_dic[developer] = 1
        else:
            developer_dic[developer] += 1

mario_3_dev = []
for key, val in developer_dic.items():
    if val == 3:
        mario_3_dev.append(key)
        
print(mario_3_dev)

['Nintendo, Nd Cube', 'Nintendo, Nintendo Software Technology']


PS2 플랫폼으로 출시된 게임들의 User_Score의 첨도를 구하여라

In [67]:
ans = game_sale_df.loc[(game_sale_df.Platform=='PS2')].User_Score.kurtosis()
print(ans)

2.4920035946875965


각 게임별 NA_Sales,EU_Sales,JP_Sales,Other_Sales 값의 합은 Global_Sales와 동일해야한다. 소숫점 2자리 이하의 생략으로 둘의 값의 다른경우가 존재하는데, 이러한 케이스가 몇개 있는지 확인하라

In [68]:
case = 0
for i in range(len(game_sale_df.index)):
    sale_1 = game_sale_df.NA_Sales[i]
    sale_2 = game_sale_df.EU_Sales[i]
    sale_3 = game_sale_df.JP_Sales[i]
    sale_4 = game_sale_df.Other_Sales[i]
    sale_5 = game_sale_df.Global_Sales[i]
    if sale_5 != sum([sale_1, sale_2, sale_3, sale_4]):
        case += 1
print(case)

3636


User_Count컬럼의 값이 120 이상인 게임들 중에서 User_Score의 값이 9.0이상인 게임의 수를 구하여라

In [70]:
print(len(game_sale_df.loc[(game_sale_df.User_Count >= 120) & (game_sale_df.User_Score >= 9)].index))

86


Global_Sales컬럼의 값들을 robust스케일을 진행하고 40이상인 데이터 수를 구하여라

In [73]:
np_global_sale = np.array(game_sale_df.Global_Sales).reshape(-1,1)
scale = RobustScaler()
game_sale_df['robust_global_sale'] = scale.fit_transform(np_global_sale)
ans = len(game_sale_df.loc[(game_sale_df.robust_global_sale > 40)].index)
print(ans)

6
